# Museum Visits Analysis

The data was obtained from the open data portal provided by the General Directorate of Museums. [Dataset Link](https://datos.gob.do/dataset/mi-conjunto-de-datos-museos-estadisticas)


## Data Overview


### Visit counts


In [2]:
import pandas as pd

df = pd.read_excel('./museum_visits.xlsx')
print(f'Rows: {df.shape[0]}\nColumns:\n{df.columns}')

Rows: 1025
Columns:
Index(['MUSEO', 'NIÑOS NACIONALES', 'ADULTOS NACIONALES', 'NIÑOS EXTRANJEROS',
       'ADULTOS EXTRANJEROS', 'PROFESORES', 'ESTUDIANTES', 'MES', 'AÑO'],
      dtype='object')


In [3]:
df.head(10)

,MUSEO,NIÑOS NACIONALES,ADULTOS NACIONALES,NIÑOS EXTRANJEROS,ADULTOS EXTRANJEROS,PROFESORES,ESTUDIANTES,MES,AÑO
0,MUSEO DE LA FAMILIA DOMINICANA,0,29,0,180,0,85,ENERO,2016
1,FORTALEZA DE SANTO DOMINGO,88,1052,105,1268,6,293,ENERO,2016
2,MUSEO DEL HOMBRE DOMINICANO,48,152,2,100,18,685,ENERO,2016
3,MUSEO FORTALEZA SAN FELIPE PTO PLATA,48,961,0,5476,0,54,ENERO,2016
4,MUSEO FARO A COLON,43,683,74,2892,0,21,ENERO,2016
5,MUSEO DE LAS CASAS REALES,56,297,10,3853,11,518,ENERO,2016
6,MUSEO ALCAZAR DE COLON,247,507,187,13790,0,287,ENERO,2016
7,MUSEO DE ARTE MODERNO,0,356,0,102,0,269,ENERO,2016
8,MUSEO JUAN PONCE DE LEON,0,0,0,0,0,0,ENERO,2016
9,MUSEO DE LA FAMILIA DOMINICANA,0,12,0,247,0,182,FEBRERO,2016


---

As we can see, the data is separated by museum name, month and year, separating the visit counts by national and international adults and minors, along with professors and students.

Despite the dataset lacks a column for type or target audience, we could infer which group of visitors prefers one museum type over the other.


In [4]:
count_by_museum = df[['MUSEO','NIÑOS NACIONALES','ADULTOS NACIONALES', 'NIÑOS EXTRANJEROS','ADULTOS EXTRANJEROS','PROFESORES','ESTUDIANTES']] \
  .groupby('MUSEO')\
  .agg('sum')
count_by_museum['NIÑOS TOTAL'] = count_by_museum['NIÑOS NACIONALES'] + count_by_museum['NIÑOS EXTRANJEROS']
count_by_museum['ADULTOS TOTAL'] = count_by_museum['ADULTOS NACIONALES'] + count_by_museum['ADULTOS EXTRANJEROS']
count_by_museum['TOTAL'] = count_by_museum['NIÑOS TOTAL'] + count_by_museum['ADULTOS TOTAL'] + count_by_museum['PROFESORES'] + count_by_museum['ESTUDIANTES']

count_by_museum = count_by_museum.sort_values('TOTAL', ascending=False)

count_by_museum.head(5)

,NIÑOS NACIONALES,ADULTOS NACIONALES,NIÑOS EXTRANJEROS,ADULTOS EXTRANJEROS,PROFESORES,ESTUDIANTES,NIÑOS TOTAL,ADULTOS TOTAL,TOTAL
MUSEO,,,,,,,,,
MUSEO ALCAZAR DE COLON,48219,117488,35166,853726,2079,100679,83385,971214,1157357
MUSEO DE LAS CASAS REALES,25875,55995,3591,330848,2079,88465,29466,386843,506853
MUSEO FORTALEZA SAN FELIPE PTO PLATA,12900,64435,9467,298546,2062,61138,22367,362981,448548
MUSEO FARO A COLON,9233,37684,17815,237714,3648,34533,27048,275398,340627
MUSEO DE ARTE MODERNO,26046,152184,2810,21732,11755,103767,28856,173916,318294


### Time Series Analysis

In order to create a time series analysis, we need to reformat the data in order to create a time series index based on the month and year of each row. We can start by converting the month string to a more manageable number, in order to create the dates.


In [5]:
month_dict = {
        "ENERO" : 1,
        "FEBRERO": 2,
        "MARZO": 3,
        "ABRIL": 4,
        "MAYO": 5,
        "JUNIO": 6,
        "JULIO": 7,
        "AGOSTO": 8,
        "SEPTIEMBRE": 9,
        "OCTUBRE": 10,
        "NOVIEMBRE": 11,
        "DICIEMBRE": 12
    }

In [6]:
count_by_date = df.replace({"MES": month_dict}, inplace=False)
count_by_date = count_by_date.astype({"MES": 'int64'}) #this in order to manage the column as an integer
count_by_date.dtype

ValueError: invalid literal for int() with base 10: 'ENERO ': Error while type casting for column 'MES'

We can see there are some inconsistencies with the month column. Some adjustments are required.


In [ ]:
df['MES'] = df['MES'].str.strip()
count_by_date = df.replace({"MES": month_dict}, inplace=False)
count_by_date = count_by_date.astype({"MES": 'int64'}) #this in order to manage the column as an integer
count_by_date.dtype

ValueError: invalid literal for int() with base 10: 'FERBERO': Error while type casting for column 'MES'

We can see that there are some typos as well, this could prove a bit difficult considering there could be many different types of typos for the same month. Let's check the unique values in the "MES" column


In [7]:
df['MES'].unique()

array(['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO',
       'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE',
       'ENERO ', 'FERBERO'], dtype=object)

In this case it seems the only typo is for "FERBERO". It would be more readable to replace all instances of "FERBERO" to "FEBRERO" and then make the replacement to integers, but it would mean passing through the dataframe twice. In this case I would rather replace the typo directly to the integer that corresponds to the month.


In [8]:
month_dict['FERBERO'] = 2
df['MES'] = df['MES'].str.strip()
count_by_date = df.replace({"MES": month_dict}, inplace=False)
count_by_date = count_by_date.astype({"MES": 'int64'}) #this in order to manage the column as an integer
count_by_date.dtypes

MUSEO                  object
NIÑOS NACIONALES        int64
ADULTOS NACIONALES      int64
NIÑOS EXTRANJEROS       int64
ADULTOS EXTRANJEROS     int64
PROFESORES              int64
ESTUDIANTES             int64
MES                     int64
AÑO                     int64
dtype: object

For time series, the index must be a date column. Considering we lack the day column, we can just take the last day of each month.


In [9]:
import calendar

def get_last_day_date(row : pd.Series, year_col_name : str, month_col_name : str) -> pd.Timestamp:
  year = row[year_col_name]
  month = row[month_col_name]
  day = calendar.monthrange(year, month)[1]
  return pd.Timestamp(year=year, month=month, day=day)

count_by_date['DATE'] = count_by_date.apply(lambda row: get_last_day_date(row, 'AÑO', 'MES'), axis=1)
count_by_date.head(5)

,MUSEO,NIÑOS NACIONALES,ADULTOS NACIONALES,NIÑOS EXTRANJEROS,ADULTOS EXTRANJEROS,PROFESORES,ESTUDIANTES,MES,AÑO,DATE
0,MUSEO DE LA FAMILIA DOMINICANA,0,29,0,180,0,85,1,2016,2016-01-31
1,FORTALEZA DE SANTO DOMINGO,88,1052,105,1268,6,293,1,2016,2016-01-31
2,MUSEO DEL HOMBRE DOMINICANO,48,152,2,100,18,685,1,2016,2016-01-31
3,MUSEO FORTALEZA SAN FELIPE PTO PLATA,48,961,0,5476,0,54,1,2016,2016-01-31
4,MUSEO FARO A COLON,43,683,74,2892,0,21,1,2016,2016-01-31


In [11]:
import seaborn as sns
pivot_date = count_by_date.pivot_table(index=['AÑO', 'MES'], columns=['MUSEO', 'NIÑOS NACIONALES','ADULTOS NACIONALES', 'NIÑOS EXTRANJEROS','ADULTOS EXTRANJEROS','PROFESORES','ESTUDIANTES'], aggfunc='sum', fill_value=0)


TypeError: datetime64 type does not support sum operations